In [1]:
import glob
import json
import hashlib
import pandas as pd

In [2]:
def generate_alphanumeric_id(timestamp):
    """Generate a unique alphanumeric ID from the timestamp."""
    # Convert the timestamp to a string
    timestamp_str = str(timestamp)
    # Create a hash of the timestamp
    hash_object = hashlib.md5(timestamp_str.encode())
    # Convert the hash to a hexadecimal string and use the first 8 characters as the ID
    return hash_object.hexdigest()[:10]

In [3]:
# Function to read and parse JSON files
def read_json_files(data_path):
    all_data = []
    for file_path in glob.glob(data_path):
        with open(file_path, 'r') as f:
            data = json.load(f)
            all_data.extend(data)
    return all_data

In [17]:
# Function to process messages and extract questions
def process_messages(data):
    questions = []
    question_askers = []
    question_timestamps = []
    answers = []
    answered_by = []
    answer_timestamps = []
    ids = []

    message_dict = {msg["ts"]: msg for msg in data}

    for message in data:
        if "subtype" not in message and message.get("text"):
            if "thread_ts" not in message or message["ts"] == message["thread_ts"]:
                question_id = generate_alphanumeric_id(message["ts"])
                if question_id not in ids:
                    questions.append(message["text"])
                    question_askers.append(message["user_profile"]["real_name"])
                    question_timestamps.append(message["ts"])
                    ids.append(question_id)

                    # Look for answers to the question in replies
                    answer, answer_by, answer_ts = find_answer(message, message_dict)
                    answers.append(answer)
                    answered_by.append(answer_by)
                    answer_timestamps.append(answer_ts)

    return ids, questions, question_askers, question_timestamps, answers, answered_by, answer_timestamps

In [5]:
# Function to find an answer to a question
def find_answer(message, message_dict):
    if "replies" in message:
        for reply in message["replies"]:
            reply_message = message_dict.get(reply["ts"])
            if reply_message:
                if reply_message["user_profile"]["real_name"] == "Alexey Grigorev":
                    return reply_message["text"], reply_message["user_profile"]["real_name"], reply_message["ts"]
        # If no reply from Alexey Grigorev, take the first reply
        first_reply = message_dict.get(message["replies"][0]["ts"])
        if first_reply:
            return first_reply["text"], first_reply["user_profile"]["real_name"], first_reply["ts"]
    return None, None, None

In [6]:
# Function to create a DataFrame from the extracted data
def create_dataframe(ids, questions, question_askers, question_timestamps, answers, answered_by, answer_timestamps):
    df = pd.DataFrame({
        "id": ids,
        "question": questions,
        "answer": answers,
        "question_asked_by": question_askers,
        "answered_by": answered_by,
        "question_timestamp": question_timestamps,
        "answer_timestamp": answer_timestamps
    })
    return df

In [18]:
# Main logic to process data
def main(data_path):
    data = read_json_files(data_path)
    ids, questions, question_askers, question_timestamps, answers, answered_by, answer_timestamps = process_messages(data)
    df = create_dataframe(ids, questions, question_askers, question_timestamps, answers, answered_by, answer_timestamps)
    return df

# Call the main function with the path to your data
df = main('../data/sample_data/*.json')

In [19]:
df

,id,question,answer,question_asked_by,answered_by,question_timestamp,answer_timestamp
0,ab8fda9401,heya,None,Kristian Maglasang,None,1712220048.482559,None
1,f12b482f76,Hello! nice to be here..:grinning:,None,luca pugliese,None,1712223185.959429,None
2,dbc60593fb,Hey there! So this is the channel to get updat...,Yes that's right,Bhavya Gupta,Alexey Grigorev,1712226627.805659,1712227767.099769
3,ff5f0391a9,Hello everyone :wave::skin-tone-2:,None,GÃ¼nal HÄ±nÃ§al,None,1712238631.304149,None
4,d4dcf112a5,Hi :wave: <@U01AXE0P5M3> when the course ia go...,"TBA\n\nOnce we know the date, I'll create a ch...",Tahir,Alexey Grigorev,1712310923.174359,1712311144.243969
5,8a6e3031dd,"Hello, nice to be here :smiley:\n\nGreetings f...",None,Humberto Rodriguez,None,1712589025.293289,None
6,90b20add2e,"Hello everyone, nice to join this channel! :gr...",None,murat kahraman,None,1713040186.854499,None
7,ef6f3dcf5a,"Hi <@U01AXE0P5M3>, can you please let me know ...",The course page has the requirements. What do ...,Ifra Saifi,Alexey Grigorev,1713179914.616589,1713180442.404689
